In [1]:
# Import necessary Python packages 
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import colors
import matplotlib.colors as clr
import pandas as pd
import os
from tensorflow.python.keras import layers
from tensorflow.python.keras import losses
from tensorflow.python.keras import models
from tensorflow.python.keras import metrics
from tensorflow.python.keras import optimizers

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

#need to import fiona and shape
import fiona
from fiona import Feature, Geometry
from shapely.geometry import mapping, shape

import os
import netCDF4 as nc
import matplotlib as mpl

from scipy.ndimage import binary_dilation

from rasterio.warp import reproject, Resampling
from rasterio.crs import CRS
from osgeo import gdal
import cv2

2024-04-04 17:12:34.543053: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
#Load Dataset

features_path = '/Users/francesco/Desktop/machine_learning_calving_project/dataset_cnn/features.npy' #Change in cluster
features = np.load(features_path, allow_pickle=True)

targets_path = '/Users/francesco/Desktop/machine_learning_calving_project/dataset_cnn/target.npy' #Change in cluster
targets = np.load(targets_path, allow_pickle=True)


In [3]:
targets.shape

(308, 1024, 1024, 1)

In [4]:
from sklearn.model_selection import train_test_split #3 mins

X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [5]:
y_test.shape

(62, 1024, 1024, 1)

In [6]:
from keras import layers
from keras import losses
from keras import models
from keras import metrics
from keras import optimizers

window_size = 1024
n_regions = 28
n_feature_variables = 6
n_target_variables = 1
n_years = 11

# Metrics
METRICS = [
      metrics.TruePositives(name='tp'),
      metrics.FalsePositives(name='fp'),
      metrics.TrueNegatives(name='tn'),
      metrics.FalseNegatives(name='fn'),
      metrics.BinaryAccuracy(name='accuracy'),
      metrics.Precision(name='precision'),
      metrics.Recall(name='recall'),
      metrics.AUC(name='auc'),
      metrics.AUC(name='prc', curve='PR')]

Input_shapes = (window_size,window_size,n_feature_variables)
optimizer = 'ADAM'
loss = 'binary_crossentropy'

In [7]:
class attention_unet():
    def __init__(self, input_shape, optimizer, loss):
        self.input_shape = input_shape
        self.optimizer = optimizer
        self.loss = loss
        self.df = 64
        self.uf = 64

    def build_unet(self):

        #Convolution Block
        def conv2d(layer_input, filters, dropout_rate=0, kernel_size=3, activation='relu',bn = False):
            d = layers.Conv2D(filters, kernel_size, strides = (1,1), padding='same')(layer_input)
            d = layers.Activation(activation)(d)
            d = layers.Conv2D(filters, kernel_size, strides = (1,1), padding='same')(d)
            d = layers.Activation(activation)(d)

            if dropout_rate:
                d = layers.Dropout(dropout_rate)(d)

            return d
        
        #Deconvolution Block
        def deconv2d(layer_input, filters, kernel_size=(3,3), activation='relu', bn = False):
            u = layers.UpSampling2D((2,2))(layer_input)
            u = layers.Conv2D(filters, kernel_size, padding='same', strides = (1,1))(u)
            u = layers.Activation(activation)(u)
            return u
        
        #Attention Block
        def attention_block(F_g, F_l, F_int, bn = False):
            g1 = layers.Conv2D(F_int, kernel_size = (1,1), strides = (1,1), padding = 'valid')(F_g)
            x1 = layers.Conv2D(F_int, kernel_size = (1,1), strides = (1,1), padding = 'valid')(F_l)
            psi = layers.Add()([g1,x1])
            psi = layers.Activation('relu')(psi)
            psi = layers.Conv2D(1, kernel_size = (1,1), strides = (1,1), padding = 'valid')(psi)
            psi = layers.Activation('sigmoid')(psi)
            return layers.Multiply()([F_l,psi])
        
        #Model
        input_layer = layers.Input(shape = self.input_shape)

        #Encoder
        conv1 = conv2d(input_layer,self.df)
        pool1 = layers.MaxPooling2D(pool_size=(2,2))(conv1)

        conv2 = conv2d(pool1, self.df*2, bn = True)
        pool2 = layers.MaxPooling2D(pool_size=(2,2))(conv2)

        conv3 = conv2d(pool2, self.df*4, bn = True)
        pool3 = layers.MaxPooling2D(pool_size=(2,2))(conv3)

        conv4 = conv2d(pool3, self.df*8, bn = True, dropout_rate = 0.5)
        pool4 = layers.MaxPooling2D(pool_size=(2,2))(conv4)

        conv5 = conv2d(pool4, self.df*16, bn = True, dropout_rate = 0.5)

        #Decoder
        up6 = deconv2d(conv5, self.uf*8, bn = True)
        deconv6 = attention_block(up6, conv4, self.uf*8, bn = True)
        uconv6 = layers.concatenate([up6, deconv6])

        up7 = deconv2d(uconv6, self.uf*4, bn = True)
        deconv7 = attention_block(up7, conv3, self.uf*4, bn = True)
        uconv7 = layers.concatenate([up7, deconv7])
        conv7 = conv2d(uconv7, self.uf*4)

        up8 = deconv2d(conv7, self.uf*2, bn = True)
        deconv8 = attention_block(up8, conv2, self.uf*2, bn = True)
        uconv8 = layers.concatenate([up8, deconv8])
        conv8 = conv2d(uconv8, self.uf*2)

        up9 = deconv2d(conv8, self.uf, bn = True)
        deconv9 = attention_block(up9, conv1, self.uf, bn = True)
        uconv9 = layers.concatenate([up9, deconv9])
        conv9 = conv2d(uconv9, self.uf)

        outputs = layers.Conv2D(1, kernel_size = (1,1), strides = (1,1), activation = 'sigmoid')(conv9)

        model = keras.Model(inputs = input_layer, outputs = outputs)

        model.compile(optimizer = self.optimizer, loss = self.loss, metrics = ['accuracy'])

        return model

In [8]:
m = attention_unet(Input_shapes, optimizer, loss).build_unet()

In [9]:
m.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 1024, 1024, 6)]      0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 1024, 1024, 64)       3520      ['input_1[0][0]']             
                                                                                                  
 activation (Activation)     (None, 1024, 1024, 64)       0         ['conv2d[0][0]']              
                                                                                                  
 conv2d_1 (Conv2D)           (None, 1024, 1024, 64)       36928     ['activation[0][0]']          
                                                                                              

 activation_11 (Activation)  (None, 128, 128, 512)        0         ['add[0][0]']                 
                                                                                                  
 conv2d_13 (Conv2D)          (None, 128, 128, 1)          513       ['activation_11[0][0]']       
                                                                                                  
 activation_12 (Activation)  (None, 128, 128, 1)          0         ['conv2d_13[0][0]']           
                                                                                                  
 multiply (Multiply)         (None, 128, 128, 512)        0         ['dropout[0][0]',             
                                                                     'activation_12[0][0]']       
                                                                                                  
 concatenate (Concatenate)   (None, 128, 128, 1024)       0         ['activation_10[0][0]',       
          

In [10]:
from tensorflow import keras
from tensorflow.keras import layers

def unet_model(input_shape=(window_size, window_size, n_feature_variables)):
    inputs = keras.Input(shape=input_shape)

    # Encoder
    conv1 = layers.Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = layers.Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = layers.Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = layers.Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)

    # Bottleneck
    conv3 = layers.Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = layers.Conv2D(256, 3, activation='relu', padding='same')(conv3)

    # Decoder
    up4 = layers.UpSampling2D(size=(2, 2))(conv3)
    concat4 = layers.Concatenate(axis=-1)([conv2, up4])
    conv4 = layers.Conv2D(128, 3, activation='relu', padding='same')(concat4)
    conv4 = layers.Conv2D(128, 3, activation='relu', padding='same')(conv4)

    up5 = layers.UpSampling2D(size=(2, 2))(conv4)
    concat5 = layers.Concatenate(axis=-1)([conv1, up5])
    conv5 = layers.Conv2D(64, 3, activation='relu', padding='same')(concat5)
    conv5 = layers.Conv2D(64, 3, activation='relu', padding='same')(conv5)

    # Output layer
    outputs = layers.Conv2D(1, 1, activation='sigmoid')(conv5)

    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

# Create the U-Net model
model = unet_model()

In [11]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 1024, 1024, 6)]      0         []                            
                                                                                                  
 conv2d_33 (Conv2D)          (None, 1024, 1024, 64)       3520      ['input_2[0][0]']             
                                                                                                  
 conv2d_34 (Conv2D)          (None, 1024, 1024, 64)       36928     ['conv2d_33[0][0]']           
                                                                                                  
 max_pooling2d_4 (MaxPoolin  (None, 512, 512, 64)         0         ['conv2d_34[0][0]']           
 g2D)                                                                                       

In [12]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [13]:
model.fit(X_train, y_train, epochs=2, batch_size=10, validation_split=0.2, verbose=1)


Epoch 1/2


: 